In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
from keras.models import Sequential
from keras.utils import to_categorical
from keras import layers
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [28]:

images = []
labels = []
for i in glob.glob('/Users/manohar/Desktop/chars74k-lite/*'):
    #print(i)
    for file in glob.glob(i+'/*.jpg'):
        image = cv2.imread(file)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        image = cv2.fastNlMeansDenoising(image)
        image = cv2.resize(image,None,fx=3.2, fy=3.2, interpolation = cv2.INTER_CUBIC)
        image = cv2.GaussianBlur(image,(5,5),0)
        T = mahotas.thresholding.otsu(image)
        img = image.copy()
        img[img>T]=255
        img[img<255]=0
        img=cv2.bitwise_not(img)  
#        processed_img = cv2.resize(img,(64,64))
#         processed_img = img.astype(np.float32)
#         for c in range(3):
#             processed_img[:,:,c] /= np.max(processed_img[:,:,c])
#         processed_img = cv2.cvtColor((processed_img*255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
#         processed_img = np.expand_dims(processed_img, -1)
#        processed_img = hog(processed_img)
        images.append(img)
        labels.append(file[39])

In [29]:
%matplotlib inline
from matplotlib import pyplot as plt

def plot_training_score(history):
  print('Availible variables to plot: {}'.format(history.history.keys()))
  for key in history.history.keys():
    print(key)
    plt.plot(history.history[key])
    plt.title('model ' + key)
    plt.ylabel(key)
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [30]:
imarray = np.asarray(images)
uniq_labels = list(set(labels))
labels_int = [uniq_labels.index(i) for i in labels]

x_train, x_test, y_train, y_test = train_test_split(imarray, labels_int, shuffle=True, random_state=42)

# Split x_train_entire and y_train_entire into training and validation

x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, shuffle=True, random_state=42)

shape_train = x_train.shape
shape_validate = x_validate.shape
shape_test = x_test.shape

# TODO: normalize the data

# Flatten all images (Both training and testing images)
x_train_flatten = x_train.reshape(shape_train[0], shape_train[1], shape_train[2], 1)
x_train_flatten = x_train_flatten.astype('float32')/255

x_val_flatten = x_validate.reshape(shape_validate[0], shape_validate[1], shape_validate[2], 1)
x_val_flatten = x_val_flatten.astype('float32')/255

x_test_flatten = x_test.reshape(shape_test[0], shape_test[1], shape_test[2], 1)
x_test_flatten = x_test_flatten.astype('float32')/255

# Convert class vectors to binary class matrices (one-hot encoding)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_validate = to_categorical(y_validate)

datagen = ImageDataGenerator(
    rotation_range=20.,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.4,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True)

datagen.fit(x_train_flatten)

# TODO: Create the model
def conv_model(img_width, img_height, channels):
    model = Sequential()  # Initalize a new model
    
    model.add(layers.Conv2D(32, (3,3),activation='relu', input_shape=(img_width, img_height, channels) ))
    model.add(layers.Conv2D(64, (3,3),activation='relu'))
    model.add(layers.Conv2D(128, (3,3),activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.35))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(26, activation='softmax'))  
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

model = conv_model(64,64,1)
# TODO: Train the model, remeber to create a validation set

#history = model.fit(x_train_flatten, y_train, epochs = 60, batch_size=32, validation_data = (x_val_flatten, y_validate))
model.fit_generator(datagen.flow(x_train_flatten, y_train, batch_size=32), steps_per_epoch = int(len(x_train)/32),epochs=20,validation_data = (x_val_flatten, y_validate), validation_steps=len(x_val_flatten)/32)
# Plot the training using the helper function created in task 0

plot_training_score(history)



# TODO: Evaluate your model and print the score of the test set

test_loss, test_acc = model.evaluate(x_test_flatten, y_test)
score = test_acc

print(score)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 62, 62, 32)        320       
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 60, 60, 64)        18496     
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 58, 58, 128)       73856     
_________________________________________________________________
flatten_12 (Flatten)         (None, 430592)            0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 430592)            0         
_________________________________________________________________
dense_23 (Dense)             (None, 256)               110231808 
_________________________________________________________________
dense_24 (Dense)             (None, 26)                6682      
Total para

KeyboardInterrupt: 

In [18]:
x_train_flatten.shape

(4000, 64, 64, 1)